<a href="https://colab.research.google.com/github/leandroaguazaco/data_science_portfolio/blob/main/Projects/04-Churn_Telco_Analysis/04_Churn_Telco_Analysis_03_modeling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<h1 align="center"> 4 - CHURN TELCO ANALYSIS </h1>

<div align="center">

  <img alt="Static Badge" src="https://img.shields.io/badge/active_project-true-blue">

</div>  

<h2 align="center"> 4.3 - Modeling </h2>

<div align="center">

  <img alt="Static Badge" src="https://img.shields.io/badge/active_section-true-blue">

  <img alt="Static Badge" src="https://img.shields.io/badge/section_status-in progress-green">

</div>  

<object
data="https://img.shields.io/badge/contact-Felipe_Leandro_Aguazaco-blue?style=flat&link=https%3A%2F%2Fwww.linkedin.com%2Fin%2Ffelipe-leandro-aguazaco%2F">
</object>

## a. Project summary

The aim of this project is to analyze and predict customer churn in the telco industry. The information pertains to client behavior, including in-call, out-call, and internet service consumption. There is a variable called 'Churn' that determines whether a customer churned within two weeks after canceling services. The information summarizes eight weeks of data for each telco line or client.

<h3 align="center"> <font color='orange'>NOTE: The project is distributed across multiple sections, separated into notebook files, in the following way:</font> </h3>



4.1 - Preprocessig data: load, join and clean data, and Exploratory data analysis, EDA.

4.2 - Premodeling: predict customer churn based on PyCaret library.

> <font color='gray'>  4.3 - Modeling: predict customer churn based on sklearn pipelines. </font> ✍ ▶ <font color='orange'> Current section </font>

4.4 - Analyzing and explaining predictions.

4.5 - Detecting vulneabilities in final machine learnig model.

4.6 - Model deployment with Streamlit.

## b. Install libraries

In [ ]:
%%capture
!pip install polars
!pip install pyjanitor # Clean DataFrame
!pip install colorama
!pip install adjustText
!pip install scikit-optimize # BayesSearchCV
!pip install umap-learn # Dimension reduction
!pip install mlflow
!pip install pyngrok
!pip install xgboost
!pip install lightgbm
!pip install catboost

## c. Import libraries

In [5]:
%%capture
# c.1 Python Utilies
import pandas as pd
import polars as pl
import numpy as np
import math
import warnings
from janitor import clean_names, remove_empty
from colorama import Fore, Style
import rpy2
import shutil
import os
from google.colab import drive
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
%%capture
# c.2 Machine learning with sklearn

import mlflow
from pyngrok import ngrok
import xgboost as xgb
import lightgbm as lgb
import catboost as catlgb

  ## Sklearn
    ### Imputation
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
    ### Preporcessing
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, OrdinalEncoder, RobustScaler, PowerTransformer
from sklearn.pipeline import make_pipeline
from sklearn.compose import ColumnTransformer
from sklearn.feature_selection import VarianceThreshold
from sklearn.decomposition import PCA
    ### Models
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, ExtraTreesClassifier
from sklearn.linear_model import BayesianRidge, LogisticRegression
from sklearn.feature_selection import SelectKBest, f_classif, mutual_info_classif # Univariate feature selection
from umap import UMAP # Dimensionallity reduction and feature selection
    ### Pipeline cache
from shutil import rmtree
from tempfile import mkdtemp
from joblib import Memory
    ### Metrics for classification
from sklearn.metrics import confusion_matrix, classification_report, roc_curve, RocCurveDisplay
from sklearn.metrics import accuracy_score, precision_score, balanced_accuracy_score, recall_score, f1_score, jaccard_score ## Higher better
from sklearn.metrics import roc_auc_score, matthews_corrcoef, cohen_kappa_score
    ### Cross validation and Hyperparameter tuning
from skopt import BayesSearchCV
from skopt.space import Real, Categorical, Integer
# from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
from sklearn.model_selection import KFold, RepeatedKFold, StratifiedKFold , RepeatedStratifiedKFold, ShuffleSplit, StratifiedShuffleSplit
from sklearn.model_selection import cross_validate, cross_val_score, cross_val_predict

In [ ]:
# c.3 Setups
%matplotlib inline
plt.style.use("ggplot")
warnings.simplefilter("ignore")

## d. Custom functions

In [7]:
# d.1 dtypes conversion and memory reduce function.
def dtype_conversion(df: pd.DataFrame = None, verbose: bool = True)-> pd.DataFrame:
    """
    Summary:
      Function to dtypes conversion and save reduce memory usage; takes a DataFrame as argument, returns DataFrame.
      For more details, visit: https://towardsdatascience.com/how-to-work-with-million-row-datasets-like-a-pro-76fb5c381cdd.
      The modifications include type casting for numerical and object variables.
    Parameters:
      df (pandas.DataFrame): DataFrame containing information.
      verbose (bool, default = True): If true, display results (conversions and warnings)
    Returns:
      pandas.DataFrame: original DataFrame with dtypes conversions
      Plot original dtypes status, variable warning due high cardinality, save memory usage, final dtypes status.
    """
    # 0- Original dtypes
    # print(Fore.GREEN + "Input dtypes" + Style.RESET_ALL)
    # print(df.dtypes)
    # print("\n")
    print(Fore.RED + "Higha Cardinality, categorical features with levels > 15" + Style.RESET_ALL)

    # 1- Original memory_usage in MB
    start_mem = df.memory_usage().sum() / 1024 ** 2

    # 2- Numerical Types
    numerics = ["int8", "int16", "int32", "int64", "float16", "float32", "float64"]
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == "int": # First 3 characters
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)
            else:
                if (c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max):
                    df[col] = df[col].astype(np.float32)
                #elif (c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max):
                #    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)

    # 3- Categorical Types
    high_card_vars = 0
    for col in df.select_dtypes(exclude = ["int8", "int16", "int32", "int64", "float16", "float32", "float64", "datetime64[ns]"]):
        categories = list(df[col].unique())
        cat_len = len(categories)
        if cat_len >= 2 and cat_len < 15:
           df[col] = df[col].astype("category")
        else:
          high_card_vars =+ 1
          # Print hight cardinality variables, amount of levels and a sample of 50 firts categories
          print(f"Look at: {Fore.RED + col + Style.RESET_ALL}, {cat_len} levels = {categories[:50]}")
    if high_card_vars == 0:
      print(Fore.GREEN + "None" + Style.RESET_ALL)
    else:
      pass

    # 4- Final memory_usage in MB
    end_mem = df.memory_usage().sum() / 1024 ** 2
    if verbose:
        print("\n")
        print(f"{Fore.RED}Initial memory usage: {start_mem:.2f} MB{Style.RESET_ALL}")
        print(f"{Fore.BLUE}Memory usage decreased to {end_mem:.2f} MB ({ 100 * (start_mem - end_mem) / start_mem:.1f}% reduction){Style.RESET_ALL}")
        #print("\n")
        #print(Fore.GREEN + "Output dtypes" + Style.RESET_ALL)
        #print(df.dtypes)
        print("\n")

    # 5. Feature types
    print(Fore.GREEN + "Variable types" + Style.RESET_ALL)
    numerical_vars = len(df.select_dtypes(include = ["number"]).columns)
    categorical_vars = len(df.select_dtypes(include = ["category", "object"]).columns)
    datetime_vars = len(df.select_dtypes(include = ["datetime64[ns]"]).columns)
    print(f"Numerical Features: {numerical_vars}")
    print(f"Categorical Features: {categorical_vars}")
    print(f"Datetime Features: {datetime_vars}")

    return df

### d.1 - Type conversions

## 1 - Load data

In [ ]:
# 1.1 - Import manin file from Google drive

# Mount Google Drive
drive.mount('/content/drive')


# Specify the source path in Google Drive
drive_filepath = '/content/drive/MyDrive/DataScience_Portfolio/04-Churn_Telco_Analysis/'

# Specify the destination path in Colab
colab_filepath = '/content/'

# Copy the file from Google Drive to Colab
try:
  shutil.copy(src = drive_filepath + '/churn_data.txt', dst = colab_filepath + '/churn_data.txt')
except:
  pass

In [22]:
%%time
# 1.2 - Load data
churn_df = pl.read_csv(source = 'churn_data.txt',
                       has_header = True,
                       separator = "|",
                       try_parse_dates = True,
                       encoding = "utf8") \
             .drop(columns = ["SUBSCRIBER_ID"]) \
             .drop_nulls(subset = ['churn']) \
             .with_columns(pl.col(['region', 'churn', 'canal', 'bandas', 'tipo_gross_adds']).cast(pl.Categorical)) \
             .to_pandas() \
             .pipe(dtype_conversion)

Higha Cardinality, categorical features with levels > 15
Look at: region, 34 levels = ['Bogota D.C.', 'Santander', 'Antioquia', nan, 'Cundinamarca', 'Quindio', 'Valle Del Cauca', 'Arauca', 'Bolivar', 'Atlantico', 'Tolima', 'Huila', 'Meta', 'Putumayo', 'Boyaca', 'Caldas', 'Cordoba', 'Nariño', 'Magdalena', 'Risaralda', 'Cauca', 'La Guajira', 'Norte De Santander', 'Caqueta', 'Cesar', 'Guaviare', 'Sucre', 'Amazonas', 'Guainia', 'Vichada', 'Choco', 'Casanare', 'Providencia Islas', 'Vaupes']


Initial memory usage: 175.54 MB
Memory usage decreased to 87.48 MB (50.2% reduction)


Variable types
Numerical Features: 38
Categorical Features: 5
Datetime Features: 0
CPU times: user 3.6 s, sys: 1.56 s, total: 5.16 s
Wall time: 5.21 s


In [24]:
churn_df.head()

,cant_sem_datos,prom_gb_tt,prom_gb_ran,prom_%_propia,continuidad_traf,variacion_datos_8s,cons_ult_sem,contrafico,mean_minutos_voz_in,mean_llamadas_in,...,bandas,tipo_gross_adds,semanas_antiguedad,semanas_contactos_pqr,contactos_ult_semana_pqr,seg_llamadas_ofertas_ult_semana,var_seg_llamadas_ofertas_4_semanas,porc_descuento_activo,jineteo,churn
0,1.0,0.603,0.000,1.000,1.000,0.000000,0.603,1,29.511000,5.571,...,Fdd Band 28,Gross Adds Migracion,1.0,1.0,2.0,0.0,0.0,0.0,1.0,No
1,5.0,3.369,0.075,0.975,1.250,-17.405001,2.783,1,52.870998,18.875,...,Ninguna,Gross Adds Nueva,4.0,0.0,0.0,0.0,0.0,0.0,1.0,No
2,8.0,0.832,0.019,0.983,0.500,-48.924999,0.425,1,64.025002,18.875,...,Ninguna,Gross Adds Migracion,16.0,0.0,0.0,0.0,0.0,0.0,1.0,No
3,8.0,6.824,0.328,0.933,0.727,-60.625000,2.687,1,148.910004,37.000,...,Fdd Band 28,Gross Adds Portacion,11.0,0.0,0.0,0.0,0.0,0.0,1.0,No
4,8.0,0.917,0.192,0.810,0.444,65.065002,1.514,1,77.785004,15.500,...,Fdd Band 28,Gross Adds Portacion,18.0,0.0,0.0,0.0,0.0,0.0,1.0,No


## 2 - Churn prediction

### 2.1 - MLflow UI Setup

In [25]:
# 1. Set the tracking URI to the default local file system
mlflow.set_tracking_uri('file:/content/mlruns')

In [ ]:
# 2. Set an experiment name and autolog
mlflow.set_experiment("04-Churn_Telco_Analysis")
mlflow.autolog(log_input_examples = True, log_model_signatures = True)